In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('Fraud.csv')

In [3]:
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [4]:
data.tail()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.0,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.0,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.0,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.0,C2080388513,0.00,0.00,1,0
6362619,743,CASH_OUT,850002.52,C1280323807,850002.52,0.0,C873221189,6510099.11,7360101.63,1,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [6]:
data.shape

(6362620, 11)

In [7]:
data.isna().sum() #check dat is clean or not

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [8]:
data.isFraud.value_counts()

isFraud
0    6354407
1       8213
Name: count, dtype: int64

In [9]:
data.isFlaggedFraud.value_counts()

isFlaggedFraud
0    6362604
1         16
Name: count, dtype: int64

In [10]:
data = data.drop(['nameOrig','nameDest'],axis=1) #data reprocessing

In [11]:
#import Label encoder
from sklearn import preprocessing 

#Label_encoder object knows how to understand word Labels.
label_encoder = preprocessing.LabelEncoder()

data['type']= label_encoder.fit_transform(data['type'])


In [12]:
X, y = data.loc[:, data.columns != 'isFraud'], data['isFraud']

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.40,random_state=42)


In [18]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [21]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics #Import for accuracy calculations

#create a gaussian Classifier
gnb = GaussianNB()

#Train the model using the training sets
gnb.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = gnb.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9955882953877491


In [25]:
from sklearn.linear_model import LogisticRegression 
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print ("Accuracy : ", metrics.accuracy_score(y_test, y_pred))

Accuracy :  0.9991882274911907


In [27]:
# Handling missing values (assuming there are missing values)
data = data.dropna()

# Handling outliers (assuming 'amount' is a feature)
Q1 = data['amount'].quantile(0.25)
Q3 = data['amount'].quantile(0.75)
IQR = Q3 - Q1
data = data[~((data['amount'] < (Q1 - 1.5 * IQR)) | (data['amount'] > (Q3 + 1.5 * IQR)))]

In [28]:
# For Logistic Regression
feature_importance = abs(classifier.coef_[0])
feature_names = X.columns
feature_importance_dict = dict(zip(feature_names, feature_importance))
sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)
print("Feature Importance:")
print(sorted_features)


Feature Importance:
[('newbalanceOrig', 34.134583340549874), ('oldbalanceOrg', 30.772245796910976), ('newbalanceDest', 12.390881826838491), ('oldbalanceDest', 11.460932762045095), ('isFlaggedFraud', 6.314501142905162), ('amount', 0.8620503867913251), ('step', 0.7529076952464114), ('type', 0.1432156940681354)]


In [29]:
from sklearn.metrics import confusion_matrix, classification_report

# For Gaussian Naive Bayes
print("Confusion Matrix for Gaussian Naive Bayes:")
print(confusion_matrix(y_test, gnb.predict(X_test)))
print("\nClassification Report for Gaussian Naive Bayes:")
print(classification_report(y_test, gnb.predict(X_test)))

# For Logistic Regression
print("Confusion Matrix for Logistic Regression:")
print(confusion_matrix(y_test, classifier.predict(X_test)))
print("\nClassification Report for Logistic Regression:")
print(classification_report(y_test, classifier.predict(X_test)))


Confusion Matrix for Gaussian Naive Bayes:
[[2533468    8314]
 [   2914     352]]

Classification Report for Gaussian Naive Bayes:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   2541782
           1       0.04      0.11      0.06      3266

    accuracy                           1.00   2545048
   macro avg       0.52      0.55      0.53   2545048
weighted avg       1.00      1.00      1.00   2545048

Confusion Matrix for Logistic Regression:
[[2541639     143]
 [   1923    1343]]

Classification Report for Logistic Regression:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   2541782
           1       0.90      0.41      0.57      3266

    accuracy                           1.00   2545048
   macro avg       0.95      0.71      0.78   2545048
weighted avg       1.00      1.00      1.00   2545048



In [31]:
#2. Describe your fraud detection model in elaboration.

# Gaussian Naive Bayes Model:
# The Gaussian Naive Bayes model is a probabilistic classification algorithm based on Bayes' theorem. 
# It assumes that features are conditionally independent given the class label, which makes it particularly suitable for datasets with a large number of features. 
# In the context of fraud detection, where the presence or absence of certain transaction features may influence the likelihood of fraud, the independence assumption
# can simplify the modeling process. This model is well-suited for situations where data is not too complex and there is a need for a fast and interpretable algorithm.

# Logistic Regression Model:
# Logistic Regression is a widely used binary classification algorithm that models the probability of an event occurring as a logistic function.
# It is well-suited for problems with a binary outcome, making it suitable for fraud detection where the goal is to classify transactions as either fraudulent or non-fraudulent.
# Logistic Regression provides interpretable outputs that indicate the contribution of each feature to the likelihood of fraud.
# It is robust, handles linear relationships well, and is less prone to overfitting, making it a valuable tool for fraud detection in financial transactions.

In [33]:
#3. How did you select variables to be included in the model?

#The features (X) are defined as all columns in the DataFrame except the target variable ('isFraud').
#The target variable (y) is set to 'isFraud'. This step is standard when preparing data for machine learning.
#The data is split into training and testing sets using the train_test_split function.

In [35]:
#4. Demonstrate the performance of the model by using best set of tools

# Importing libraries
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# ... ( existing code for model training and scaling)

# Gaussian Naive Bayes Model
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_gnb = gnb.predict(X_test)

# Evaluate Gaussian Naive Bayes model
accuracy_gnb = accuracy_score(y_test, y_pred_gnb)
conf_matrix_gnb = confusion_matrix(y_test, y_pred_gnb)
class_report_gnb = classification_report(y_test, y_pred_gnb)

print("Gaussian Naive Bayes Model Performance:")
print("Accuracy:", accuracy_gnb)
print("\nConfusion Matrix:")
print(conf_matrix_gnb)
print("\nClassification Report:")
print(class_report_gnb)

# Logistic Regression Model
classifier = LogisticRegression(random_state=0)
classifier.fit(X_train, y_train)
y_pred_logreg = classifier.predict(X_test)

# Evaluate Logistic Regression model
accuracy_logreg = accuracy_score(y_test, y_pred_logreg)
conf_matrix_logreg = confusion_matrix(y_test, y_pred_logreg)
class_report_logreg = classification_report(y_test, y_pred_logreg)

print("\nLogistic Regression Model Performance:")
print("Accuracy:", accuracy_logreg)
print("\nConfusion Matrix:")
print(conf_matrix_logreg)
print("\nClassification Report:")
print(class_report_logreg)

Gaussian Naive Bayes Model Performance:
Accuracy: 0.9955882953877491

Confusion Matrix:
[[2533468    8314]
 [   2914     352]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   2541782
           1       0.04      0.11      0.06      3266

    accuracy                           1.00   2545048
   macro avg       0.52      0.55      0.53   2545048
weighted avg       1.00      1.00      1.00   2545048


Logistic Regression Model Performance:
Accuracy: 0.9991882274911907

Confusion Matrix:
[[2541639     143]
 [   1923    1343]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   2541782
           1       0.90      0.41      0.57      3266

    accuracy                           1.00   2545048
   macro avg       0.95      0.71      0.78   2545048
weighted avg       1.00      1.00      1.00   2545048



In [40]:
#5. What are the key factors that predict fraudulent customer? 

#Interpretation of key factors should consider the nature of the features. For example, if 'amount' is an important factor, a higher amount might indicate a higher risk of fraud. 
#Similarly, if 'type' is a crucial factor, certain transaction types might be more associated with fraud.

#a. Logistic Regression Model:
#Logistic Regression provides coefficients for each feature, indicating the log-odds of the target variable (fraud) based on changes in the corresponding feature.
#Positive coefficients suggest an increase in the likelihood of fraud, while negative coefficients suggest a decrease.
#The larger the absolute value of the coefficient, the more impactful the feature is in predicting fraud.

#b. Gaussian Naive Bayes Model:
#Gaussian Naive Bayes does not provide direct feature importance, but we can still analyze the distribution and characteristics of features to understand their impact.
#Look for features with notable differences in distribution between fraudulent and non-fraudulent transactions.


In [37]:
#6. Do these factors make sense? If yes, How? If not, How not? 

#Yes these factors make sense to a certain extent:
#Amount: A positive coefficient for 'amount' may suggest that larger transaction amounts are more likely to be associated with fraud, which aligns
#with the common understanding that fraudulent transactions often involve higher sums.

#Transaction Type ('type'): If 'type' is a significant factor, different transaction types may have varying associations with fraud. 
#For example, certain types like 'TRANSFER' or 'CASH_OUT' might be more indicative of fraud.

In [38]:
#7. What kind of prevention should be adopted while company update its infrastructure?

#Securing the data handling process:
#data protection by adopting end-to-end encryption for financial transactions and sensitive customer information

#continuous/real time monitoring:
#Enhance real-time monitoring of user activities and transactions to quickly detect anomalies or unusual patterns that may indicate fraudulent behavior.

#ML based fraud detection models:
#Use machine learning models for fraud detection and continuously updating and refining them as the infrastructure evolves.
#Incorporate insights gained from model interpretation to improve accuracy.

#Setting up dynamic limits: 
#Implement adaptive transaction monitoring with dynamic methods, adjusting limits based on historical data and emerging fraud patterns.

#last but not the least:
#By creating awareness among employees and customers.

In [39]:
#8. Assuming these actions have been implemented, how would you determine if they work?

#Performances checks:
#Accuracy and Precision: check for the accuracy of the fraud detection models, ensuring that they accurately classify transactions.
#Evaluate precision, which measures the proportion of predicted fraud cases that are actually fraudulent.
#False Positive Rate: Assess the false positive rate to minimize the number of legitimate transactions incorrectly flagged as fraud.

#Fraud Incidents and user experiences: Track the number of actual fraud incidents over time. A reduction in the frequency of fraud incidents may indicate the
# effectiveness of preventive measures and also take feedbacks from users

#Results analysis: Review the results of security audits and penetration tests. Successful tests indicate that the infrastructure can withstand simulated attacks.

#Response Time: Assess the response time and effectiveness of the incident response plan during actual security incidents.
# A well-executed response minimizes the impact of security breaches.

#Adaptability of the model: Assess the system's ability to adapt to new and emerging fraud patterns. Regularly update the fraud detection models to address evolving threats.